Теперь посмотрим как семантические вектора помогают решать задачу Word Sense Induction - WSI.

Для начала скопируем из предыдущей тетрадки все полезное: загрузим модель и контексты, определим токенизатор и функцию векторного представления контекста.

In [8]:
import re
import numpy as np
from gensim.models import Word2Vec
from pymystem3 import Mystem
import rl_wsd_labeled

mystem = Mystem()

In [9]:
w2v_model = Word2Vec.load('../rlwsd/models/w2v.pkl')

In [4]:
_, contexts = rl_wsd_labeled.get_contexts(
    rl_wsd_labeled.contexts_filename('nouns', 'RuTenTen', 'горшок'))

In [7]:
def tokenize(s):
    return [t for t in mystem.lemmatize(s)
            if re.match('\w+$', t)]

def context_repr(context):
    left, _, right = context
    words = tokenize(left) + tokenize(right)
    return np.mean([w2v_model[w] for w in words if w in w2v_model],
                    axis=0)

context_repr(contexts[1][0])

array([-0.02453865,  0.00159344,  0.0526606 , ...,  0.01031213,
       -0.01585422,  0.02414869], dtype=float32)

Подготовим данные для кластеризации:

In [19]:
xs = [ctx for ctx, _ in contexts]
ys = np.array([int(s) - 1 for _, s in contexts])
xs_vec = np.array([context_repr(ctx) for ctx in xs])

Попробуем использовать для кластеризации ``KMeans``:

In [14]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=6)
kmeans.fit(xs_vec)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

Результат кластеризации k-means это цетры кластеров:

In [15]:
kmeans.cluster_centers_

array([[-0.02604795, -0.03269598,  0.01746345, ...,  0.02113272,
         0.03147279,  0.03588657],
       [-0.01671131, -0.02545614,  0.04128313, ...,  0.00794684,
         0.01390636,  0.05033641],
       [-0.00174833, -0.02311012,  0.02854693, ...,  0.00688501,
         0.02628346,  0.05596653],
       [-0.01390544, -0.00363304,  0.02211228, ..., -0.00166642,
         0.02305105,  0.03536921],
       [ 0.01196016, -0.03844162,  0.05034662, ...,  0.0379402 ,
        -0.02400529,  0.04493199],
       [ 0.00523113, -0.0384718 ,  0.07009128, ...,  0.00158443,
         0.00127588,  0.0548449 ]], dtype=float32)

Также у нас уже есть номера кластеров для всех контекстов:

In [16]:
kmeans.labels_

array([5, 2, 0, 2, 0, 1, 2, 3, 3, 2, 3, 3, 3, 3, 2, 4, 4, 3, 3, 2, 3, 3, 3,
       3, 0, 3, 3, 3, 3, 2, 3, 0, 3, 3, 3, 3, 3, 2, 3, 0, 2, 3, 0, 2, 4, 1,
       3, 2, 0, 0, 2, 2, 1, 2, 3, 2, 4, 2, 3, 3, 3, 3, 2, 3, 0, 3, 2, 0, 5,
       2, 0, 3, 0, 2, 0, 0, 0, 2, 3, 3, 3, 2, 4, 3, 4, 3, 3, 3, 3, 3, 2, 4,
       0, 2, 1, 2, 3, 3, 3, 2, 0, 1, 3, 3, 2, 3, 1, 3, 3, 3, 0, 2, 0, 3, 2,
       1, 3, 2, 2, 2, 3, 3, 4, 4, 2, 3, 0, 3, 4, 5, 2, 0, 1, 2, 3, 3, 1, 2,
       2, 2, 3, 1, 1, 3, 3, 3, 3, 3, 1, 3, 1, 1, 3, 3, 2, 3, 2, 2, 4, 1, 2,
       3, 1, 0, 0, 2, 4, 4, 1, 3, 3, 3, 1, 0, 3, 3, 0, 3, 2, 3, 3, 1, 0, 1,
       4, 4, 0, 1, 3, 3, 3, 3, 1, 3, 2, 3, 3, 3, 0, 1, 1, 1, 2, 3, 3, 0, 2,
       2, 0, 3, 3, 3, 3, 2, 3, 1, 2, 2, 2, 1, 3, 1, 3, 5, 2, 3, 3, 3, 2, 2,
       1, 2, 2, 1, 3, 1, 3, 2, 2, 2, 0, 2, 1, 2, 4, 3, 3, 1, 3, 1, 3, 2, 1,
       3, 2, 2, 0, 3, 3, 3, 3, 3, 3, 4, 5, 3, 3, 2, 3, 2, 3, 2, 1, 3, 5, 3,
       2, 2, 1, 2, 0, 2, 2, 3, 3, 3, 2, 2, 3, 3, 2, 3, 1, 0, 1, 3, 4, 1, 3,
       2, 3,

Как оценить точность кластеризации?
Мы не можем просто сравнить номера размеченных и найденных значений - мы можем только сравнить,
насколько кластеризация похожа на "истинную":

In [20]:
from sklearn.metrics import adjusted_rand_score

adjusted_rand_score(ys, kmeans.labels_)

0.091021874246701939

Задания:

Непонятно, что за значения выделил метод, насколько они осмысленные?
Попробуйте представить выделенные значения:
 - через ближайшие слова
 - через типичные контексты
 
Как еще можно визуализировать работу метода и качество представлений значений? Попробуйте использовать t-SNE.